# Adaptive PDE discretizations on cartesian grids 
## Volume : Algorithmic tools
## Part : Automatic differentiation
## Chapter : Known bugs and incompatibilities

The techniques of automatic differentiation technique play an essential role in the notebooks presented in this repository. 

[**Summary**](Summary.ipynb) of volume Algorithmic tools, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1 Matrix multiplication and inversion](#1-Matrix-multiplication-and-inversion)
  * [2. In place modifications and aliasing](#2.-In-place-modifications-and-aliasing)
    * [2.1 Aliasing of the AD information](#2.1-Aliasing-of-the-AD-information)
    * [2.2 Non writeable AD information](#2.2-Non-writeable-AD-information)



**Acknowledgement.** The experiments presented in these notebooks are part of ongoing research, 
some of it with PhD student Guillaume Bonnet, in co-direction with Frederic Bonnans.

Copyright Jean-Marie Mirebeau, University Paris-Sud, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"..") # Allow importing agd from parent directory
#from Miscellaneous import TocTools; TocTools.displayTOC('ADBugs','Algo')

In [2]:
import numpy as np
import scipy.sparse.linalg

In [3]:
import agd.AutomaticDifferentiation as ad

In [4]:
def reload_packages():
    from Miscellaneous.rreload import rreload
    global ad
    ad, = rreload([ad],rootdir='..',verbose=True)

## 1 Matrix multiplication and inversion

Please use the `ad.apply_linear_mapping` and `ad.apply_linear_inverse` functions in combination with `np.dot`, or scipy solve functions.

In [36]:
v = ad.Dense.denseAD( np.random.standard_normal((4,)),np.random.standard_normal((4,4)))
m0 = np.random.standard_normal((4,4))
m1 = scipy.sparse.coo_matrix( ([1.,2.,3.,4.,5.],([0,2,1,2,3],[0,1,2,2,3]))).tocsr()

In [37]:
# Fails
#print("np.dot looses AD:",np.dot(m0,v))
#print("scipy '*' looses AD:",m1*v) 

In [38]:
print("np.dot with AD:\n",ad.apply_linear_mapping(m0,v))
print("scipy '*' with AD:\n",ad.apply_linear_mapping(m1,v))

np.dot with AD:
 denseAD([ 2.35235392  0.26440333  2.24259106 -1.68842825],
[[-2.98560581  1.30578452  0.4219539   0.4405235 ]
 [-0.90830923 -2.40696381  2.22900244 -1.97361014]
 [-3.73570982 -0.18563747  1.75156844  1.11318797]
 [ 0.17856435 -2.59378054  0.39560053  2.40460541]])
scipy '*' with AD:
 denseAD([ 0.56140742 -0.37699399 -4.1332458   8.15577671],
[[-1.45923092e+00 -9.81010117e-02 -5.06670857e-01  7.69283240e-01]
 [ 2.43968361e-03 -4.08462334e-01  9.69397632e-01  1.93501014e+00]
 [ 1.35342886e+00 -5.74825397e+00  1.50053419e+00  5.08502685e+00]
 [-9.60842086e+00  4.62957060e-01  6.52340297e+00 -5.02911061e+00]])


In [39]:
print("scipy solve with AD :\n",ad.apply_linear_inverse(scipy.sparse.linalg.spsolve,m1,v))

scipy solve with AD :
 denseAD([ 0.56140742  1.14736338 -0.60509786  0.32623107],
[[-1.45923092 -0.09810101 -0.50667086  0.76928324]
 [-0.44965203  1.66646878  0.0922316  -0.51250275]
 [ 0.22502932 -0.86727292  0.03466734  0.41750222]
 [-0.38433683  0.01851828  0.26093612 -0.20116442]])


## 2. In place modifications and aliasing

The AD information often consists of very large arrays. In order to save time and memory, this information is not systematically copied and/or stored fully. It can take the form of a broadcasted array, or of an alias to another array. In that case a copy is necessary to enable modifications.

### 2.1 Aliasing of the AD information

When an operation leaves the AD information untouched, an alias is used. This can lead to bugs if in place modifications are used afterward.

In [40]:
x=ad.Dense.identity(constant=np.array([1.,2.]))
y=x+1 # Only affects the value, not the AD information

In [41]:
print("Values are distinct :", x.value is y.value)
print("AD information is shared :", y.coef is x.coef)

Values are distinct : False
AD information is shared : True


A modification of the aliased variable will impact the original one.

In [42]:
print(x[0])
y[0]*=2
print("Caution ! Shared AD information is affected :", x[0])

denseAD(1.0,[1. 0.])
Caution ! Shared AD information is affected : denseAD(1.0,[2. 0.])


Avoid this effect by making a copy.

In [43]:
x=ad.Dense.identity(constant=np.array([1.,2.]))
y=(x+1).copy()
print("AD information is distinct :", y.coef is x.coef)

AD information is distinct : False


Note that a similar effect arises with the `-` binary operator, but not with `*`or `/`. That is because the latter modify the AD information, which therefore must be copied anyway.

In [44]:
x=ad.Dense.identity(constant=np.array([1.,2.]))
print("AD information is shared :", (x-1).coef is x.coef)
print("AD information is distinct :", (x*2).coef is x.coef)
print("AD information is distinct :", (x/2).coef is x.coef)

AD information is shared : True
AD information is distinct : False
AD information is distinct : False


### 2.2 Non writeable AD information

When creating an dense AD variable, the coefficients may be non writeable (e.g. broadcasted) arrays.

In [45]:
x=ad.Dense.identity(constant=np.array([[1.,2.],[3.,4.]]),shape_bound=(2,))

In [46]:
x.coef.flags.writeable

False

In [47]:
# x+=1 # Fails because non-writeable

Make a copy to solve the issue.

In [48]:
y=x.copy()

In [49]:
y.coef.flags.writeable

True

In [50]:
y+=1